In [1]:
import os
import sys
import tensorflow as tf
import numpy as np

from tensorflow import keras

import pandas as pd
from d2l import tensorflow as d2l
import csv


sys.path.append("/home/hpczeji1/hpc-work/Codebase/AllAbout_DeepLearning/Tutorial/my_tf_pipline/")
from utils.dataset import download,download_all,download_extract,get_kaggle_house_dataset
from utils.dataset_loader import load_cifar10,  id_table_dataset_loader
from backbone.cnn import CNN_3L, Sequential_CNN_3L
from backbone.mlp import Stacked_Regression_MLP, Sequential_Regression_MLP,get_Sequential_Regression_MLP

In [2]:
# 1&2 dataset load and data preprocess
train_data, test_data = get_kaggle_house_dataset() #! noramlly you can use tf.convert_to_tensor(train_features, dtype = tf.float32) to process data to tensor
print(f"train&test shape: {train_data.shape, test_data.shape}")
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]) # visualize the first4 and last 2 features
train_features, test_features, train_labels, n_train = id_table_dataset_loader(train_data, test_data)
# dataloader,(x_train, y_train), (x_test, y_test)  =  load_cifar10(BATCH_SIZE = 64, buffer_size=1024)


train&test shape: ((1460, 81), (1459, 80))
   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000
train and test shape:(TensorShape([1460, 331]), TensorShape([1459, 331]))


In [3]:
# # 3 model 

model = get_Sequential_Regression_MLP() # Sequential_Regression_MLP() #  Sequential_CNN_3L(input_shape = (32,32,3)) 

# model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

# # 4&5 set optimizer and train
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer="Adam", loss = tf.keras.losses.mean_squared_logarithmic_error) # ,metrics=['accuracy'] # Kers have log version of MeanSquaredError 

# reshaped_x_train = x_train.reshape(50000, 32, 32, 3).astype("float32") / 255
model.fit(train_features , train_labels, epochs=200,batch_size=32,verbose=0)
model.summary()

Epoch 1/100
46/46 [==============================] - 0s 1ms/step - loss: 128.8116
Epoch 2/100
46/46 [==============================] - 0s 957us/step - loss: 115.4030
Epoch 3/100
46/46 [==============================] - 0s 905us/step - loss: 109.0132
Epoch 4/100
46/46 [==============================] - 0s 769us/step - loss: 104.9005
Epoch 5/100
46/46 [==============================] - 0s 788us/step - loss: 101.8067
Epoch 6/100
46/46 [==============================] - 0s 807us/step - loss: 99.3142
Epoch 7/100
46/46 [==============================] - 0s 783us/step - loss: 97.2242
Epoch 8/100
46/46 [==============================] - 0s 816us/step - loss: 95.4217
Epoch 9/100
46/46 [==============================] - 0s 880us/step - loss: 93.8375
Epoch 10/100
46/46 [==============================] - 0s 826us/step - loss: 92.4223
Epoch 11/100
46/46 [==============================] - 0s 764us/step - loss: 91.1449
Epoch 12/100
46/46 [==============================] - 0s 774us/step - loss: 89.978

In [11]:

print("Start to predict...")
predictions = model.predict(test_features)
# model.evaluate() # when have labels, you can use evalute # use print(dir(tf.random/tf.model)) to see sub func
test_imgs_list = test_data.iloc[:,0] #! if shuffled, how to get testset' name order?
assert len(test_imgs_list) == len(predictions)
with open("../results/mlp_predictions_submit.csv","w") as f:
    writer = csv.writer(f, delimiter=",")
    for i in range(len(test_imgs_list)):
        writer.writerow([test_imgs_list[i],predictions[i][0]])  # writerows suitablr for writing rows in laterally/horizontally

Start to predict...


In [21]:
 #! save in pandas way is more suffient, and meet the kaggle requirements
predictions = np.array(predictions)
test_data["SalePrice"] = pd.Series(predictions.reshape(1,-1)[0]) #! same with (-1,) or (-1,1) 
# print(predictions.shape)
# print(predictions.reshape(1,-1))
submission = pd.concat([test_data["Id"], test_data["SalePrice"]],axis=1)
submission.to_csv("../results/submission.csv", index = False)
